<a href="https://colab.research.google.com/github/asra020601/sentiment_analysis_using_transfomer/blob/main/twt_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

By employing a Transformer architecture, this project aims to enhance sentiment analysis of Twitter comments by effectively capturing long-range dependencies and intricate semantic nuances within the text.


# LIBRARIES & DATASETS

Importing essential dependencies and ingesting the dataset for preprocessing and analysis

In [ ]:
! pip install datasets
! pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
 

In [ ]:
import pandas as pd
from datasets import load_dataset
import tiktoken
import torch.nn as nn
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import math
import time

In [ ]:
main_data = load_dataset("Sp1786/multiclass-sentiment-analysis-dataset")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

train_df.csv:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

val_df.csv:   0%|          | 0.00/601k [00:00<?, ?B/s]

test_df.csv:   0%|          | 0.00/586k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/31232 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5205 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5206 [00:00<?, ? examples/s]

Serializing the dictionary object into a structured CSV format for enhanced data manipulation and persistence.

In [ ]:
main_data

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'sentiment'],
        num_rows: 31232
    })
    validation: Dataset({
        features: ['id', 'text', 'label', 'sentiment'],
        num_rows: 5205
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'sentiment'],
        num_rows: 5206
    })
})

In [ ]:
train_dataset = main_data['train']
test_dataset = main_data['test']
val_dataset = main_data['validation']
# Convert to a Pandas DataFrame
train_df = train_dataset.to_pandas()
val_df = val_dataset.to_pandas()
test_df = test_dataset.to_pandas()

# Save to CSV
train_df.to_csv("train_data.csv", index=False)
val_df.to_csv("val_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)

print("Train data has been saved to train_data.csv")
print("Validation data has been saved to val_data.csv")
print("Test data has been saved to test_data.csv")

Train data has been saved to train_data.csv
Validation data has been saved to val_data.csv
Test data has been saved to test_data.csv


In [ ]:
test_df

,id,text,label,sentiment
0,9235,getting cds ready for tour,1,neutral
1,16790,"MC, happy mother`s day to your mom ;).. love yah",2,positive
2,24840,A year from now is graduation....i am pretty s...,0,negative
3,20744,because you had chips and sale w/o me,1,neutral
4,6414,Great for organising my work life balance,2,positive
...,...,...,...,...
5201,10277,Fuck no internet damn time warner!,0,negative
5202,8610,Looking forward to android 1.5 being pushed t...,1,neutral
5203,8114,Not good. Wasted time.,0,negative
5204,3034,"U were great, as always. But, can`t we do an ...",2,positive


In [ ]:
train_df = pd.read_csv('train_data.csv')
test_df = pd.read_csv('test_data.csv')
val_df = pd.read_csv('val_data.csv')

Implementing text tokenization and frequency analysis to identify the most frequently occurring terms in the corpus

In [ ]:
train_df['Word_Count'] = train_df['text'].apply(lambda x: len(x.split()))
max_word_sentence = train_df.loc[train_df['Word_Count'].idxmax()]
print("Sentence with the most words:")
print(max_word_sentence['text'])
print(f"Word count: {max_word_sentence['Word_Count']}")


Sentence with the most words:
Is it true that non-premium version soon will loose any way for backup/restore or transfer to another phone? If not true: Please point to me what is the backup/restore way that stays in app in the future. Thanks a lot!. If true: That is really bad news - backup is a basic necessity, so non-premium version will become practically useless on the long term. In addition, premium costs twice as much for a single year than, let's say OsmAnd+ for eternity. And that is a much more complex app + the free version is completely usable within reasonable limits. Tasks is a really nice app (except that swichingt between sublists/lists is unreasonably slow) with some unique and essential features. But this wracks it, it will drive me away. Possibly others too. Not a fair move to reduce base functionality like that. RESPONSE TO STEVEN NOTTAGE'S RESPONSE: That is fair enough, except for one important detail: all the people using your free app are now forced to decide - if 

# TOKENIZER

Initializing and configuring the NLP tokenizer to facilitate text segmentation and linguistic processing.

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")
print(tokenizer.encode("<|endoftext|>", allowed_special={"<|endoftext|>"}))

[50256]


In [ ]:
print(train_df["label"].value_counts())


label
1    11649
2    10478
0     9105
Name: count, dtype: int64


# DATALOADER

Implementing a custom Dataset class that generates input-target pairs, incorporating dynamic sequence truncation and padding mechanisms to ensure uniform tensor dimensions across train/validation/test splits. The sequences are truncated to max_length if they exceed the specified threshold, while shorter sequences are padded to maintain consistent batch processing.

In [ ]:
class Classification_dataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=None, pad_token_id=50256):
        self.data = pd.read_csv(csv_file)
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.pad_token_id = pad_token_id
        self.data['text'] = self.data['text'].astype(str)
        self.encoded_texts = [
            tokenizer.encode(text) for text in self.data["text"]
        ]
        if max_length is None:
            self.max_length = self._longest_encoded_length()
        else:
            self.max_length = max_length

            self.encoded_texts = [
                encoded_text[:self.max_length]
                for encoded_text in self.encoded_texts
            ]

        self.encoded_texts = [
            encoded_text + [pad_token_id] * (self.max_length - len(encoded_text))
            for encoded_text in self.encoded_texts
        ]
        self.labels = self.data["label"]

    def __getitem__(self, index):
        encoded = self.encoded_texts[index]
        label = self.data.iloc[index]["label"]
        return (
            torch.tensor(encoded, dtype=torch.long),
            torch.tensor(label, dtype=torch.long)
        )

    def __len__(self):
        return len(self.data)

    def _longest_encoded_length(self):
        max_length = 0
        for encoded_text in self.encoded_texts:
            encoded_length = len(encoded_text)
            if encoded_length > max_length:
                max_length = encoded_length
        return max_length


Dynamically setting sequence max_length based on the training dataset parameters, then instantiating our custom Dataset class to process and standardize the train, validation, and test splits accordingly.

In [ ]:
train_dataset = Classification_dataset(
    csv_file="/content/train_data.csv",
    max_length=None,
    tokenizer=tokenizer
)

print(train_dataset.max_length)

545


In [ ]:
val_dataset = Classification_dataset(
    csv_file="/content/val_data.csv",
    max_length=train_dataset.max_length,
    tokenizer=tokenizer
)
test_dataset = Classification_dataset(
    csv_file= "/content/test_data.csv" ,
    max_length=train_dataset.max_length,
    tokenizer=tokenizer
)


Initializing PyTorch DataLoader with a batch_size of 8 to efficiently stream and batch the preprocessed sequences for model training.


In [ ]:
num_workers = 0
batch_size = 8

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    drop_last=True,
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False,
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False,
)

In [ ]:
print(f"{len(train_loader)} training batches")
print(f"{len(val_loader)} validation batches")
print(f"{len(test_loader)} test batches")

3904 training batches
651 validation batches
651 test batches


In [ ]:
#just an example input
inputs = tokenizer.encode("Do you have time")
inputs = torch.tensor(inputs).unsqueeze(0)
print("Inputs:", inputs)
print("Inputs dimensions:", inputs.shape) # shape: (batch_size, num_tokens)

Inputs: tensor([[5211,  345,  423,  640]])
Inputs dimensions: torch.Size([1, 4])


# EMBEDDINGS

In [ ]:
embeddings = torch.randn(inputs.shape[0], inputs.shape[1], 32)  # Example embedding dimensions


Generating logarithmic positional encodings to capture sequential information and relative positions within the transformer architecture's self-attention mechanism

In [ ]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)


        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)

Final embeddings are computed by summing learned token embeddings with positional embeddings, ensuring compatible tensor shapes.

In [ ]:
# prompt:
# Example:
pos_embeddings = PositionalEncoding(32, 0.1)  # Example embedding dimensions
sequence_length = inputs.shape[1]  # Assuming 'inputs' has shape (batch_size, sequence_length)
final_embeddings = embeddings + pos_embeddings.pe[:, :sequence_length, :]

print("Final Embeddings Shape:", final_embeddings.shape)

Final Embeddings Shape: torch.Size([1, 4, 32])


# SELF ATTENTION


Attention masking was applied to selectively ignore irrelevant information during the attention calculation.

In [ ]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # New
        #self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New

    def forward(self, x):
        b, num_tokens, d_in = x.shape # New batch dimension b
        mask = torch.triu(torch.ones(num_tokens, num_tokens), diagonal=1).to(x.device) # Create mask based on input size

        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # Changed transpose
        attn_scores.masked_fill_(  # New, _ ops are in-place
            mask.bool(), -torch.inf)  # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # New

        context_vec = attn_weights @ values
        return context_vec


In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "Every effort moves you"
txt2 = "Every day holds a"
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch.shape)

torch.Size([2, 4])


In [ ]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

torch.Size([2, 1, 4])


In [ ]:
batch

tensor([[[5211,  345,  423,  640]],

        [[5211,  345,  423,  640]]])

In [ ]:
 inputs.shape[1]

4

In [ ]:
d_in = inputs.shape[1]
d_out = 4
context_length = inputs.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)#what is context_length?

# To this:
context_vecs = ca(batch.float())

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[ 816.0527, 2660.2507, 1609.4360, -327.7043]],

        [[ 816.0527, 2660.2507, 1609.4360, -327.7043]]],
       grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 1, 4])


A multi-attention wrapper was employed to leverage multiple attention perspectives within the model.

In [ ]:
class MultiHeadAttentionWrapper(nn.Module):

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias)
             for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [ ]:
context_length = batch.shape[1] # This is the number of tokens
d_in, d_out =  inputs.shape[1], 4
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)

context_vecs = mha(batch.float())

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[ -359.4584,   948.6454,  1195.3362, -2233.8333,   816.0060,
          -1598.8204,  1590.9561,  1558.0779]],

        [[ -359.4584,   948.6454,  1195.3362, -2233.8333,   816.0060,
          -1598.8204,  1590.9561,  1558.0779]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 1, 8])


## HELPER FUNCTIONS

To facilitate modularity and code reusability, helper functions were defined for common operations, including the GELU activation function, layer normalization, and feedforward transformations.

In [ ]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))

In [ ]:
class LayerNorm(nn.Module):
   def __init__(self, emb_dim):
       super().__init__()
       self.eps = 1e-5
       self.scale = nn.Parameter(torch.ones(emb_dim))
       self.shift = nn.Parameter(torch.zeros(emb_dim))
   def forward(self, x):
       mean = x.mean(dim=-1, keepdim=True)
       var = x.var(dim=-1, keepdim=True, unbiased=False)
       norm_x = (x - mean) / torch.sqrt(var + self.eps)
       return self.scale * norm_x + self.shift

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

The model was configured with a vocabulary size of 50,257, a context length of 4 tokens, 12 attention heads, 12 layers, a 32-dimensional embedding space, a 0.1 dropout rate, and no bias for query, key, and value projections.

In [ ]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 4, # Context length
    "emb_dim": 32,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

#TRANSFORMER

A modular transformer block was defined as a fundamental building unit, which was then stacked to construct the GPT model architecture.

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttentionWrapper(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"]// cfg["n_heads"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])
        self.proj = nn.Linear(cfg["n_heads"] * (cfg["emb_dim"] // cfg["n_heads"]), cfg["emb_dim"]) # Project back to emb_dim

    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)  # Shape [batch_size, num_tokens, emb_size]
        x = self.proj(x) # Project the output of the attention layer back to the original embedding size

        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x

In [ ]:

class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = PositionalEncoding(cfg['emb_dim'],0.1)
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        # Use a placeholder for TransformerBlock
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        # Use a placeholder for LayerNorm
        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = PositionalEncoding(tok_embeds.shape[-1],0.1)
        x = tok_embeds + pos_embeds.pe[:, :in_idx.shape[1], :]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "Every effort moves you"
txt2 = "Every day holds a"
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch.shape)

torch.Size([2, 4])


In [ ]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)

out = model(batch)
print("Input batch:\n", batch)
print("\nOutput shape:", out.shape)
print(out)

Input batch:
 tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])

Output shape: torch.Size([2, 4, 50257])
tensor([[[-0.1671, -0.5093,  0.5903,  ..., -0.9000,  0.8107,  0.8745],
         [-0.9405,  0.4408, -0.0059,  ..., -0.6058, -1.0605,  0.4361],
         [-0.7046,  0.4471,  0.5488,  ..., -0.6855, -0.4316,  0.7119],
         [ 0.0702,  1.2890,  0.5067,  ..., -0.5216,  0.0286, -0.3515]],

        [[ 0.1022, -0.2914,  0.6886,  ..., -0.8930,  0.5021,  0.1087],
         [ 0.0940, -0.3542, -0.4415,  ..., -0.1324, -0.7538,  0.7168],
         [ 0.3554,  0.5393,  0.2843,  ..., -0.3169,  0.2386, -0.1166],
         [ 0.1867,  0.3384, -0.2131,  ..., -0.4351, -1.5658,  0.2684]]],
       grad_fn=<UnsafeViewBackward0>)


In [ ]:
print("Token embedding layer shape:", model.tok_emb.weight.shape)
print("Output layer shape:", model.out_head.weight.shape)

Token embedding layer shape: torch.Size([50257, 32])
Output layer shape: torch.Size([50257, 32])


In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

Total number of parameters: 3,355,520


In [ ]:
# Calculate the total size in bytes (assuming float32, 4 bytes per parameter)
total_size_bytes = total_params * 4

# Convert to megabytes
total_size_mb = total_size_bytes / (1024 * 1024)

print(f"Total size of the model: {total_size_mb:.2f} MB")

Total size of the model: 12.80 MB


A function, generate_text_simple, was defined to generate text. It takes the model, the current context indices (idx), the maximum number of new tokens to generate (max_new_tokens), and the context size as input.

In [ ]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (batch, n_tokens) array of indices in the current context
    for _ in range(max_new_tokens):

        # Crop current context if it exceeds the supported context size
        # E.g., if LLM supports only 5 tokens, and the context size is 10
        # then only the last 5 tokens are used as context
        idx_cond = idx[:, -context_size:]

        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond)

        # Focus only on the last time step
        # (batch, n_tokens, vocab_size) becomes (batch, vocab_size)
        logits = logits[:, -1, :]

        # Apply softmax to get probabilities
        probas = torch.softmax(logits, dim=-1)  # (batch, vocab_size)

        # Get the idx of the vocab entry with the highest probability value
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)  # (batch, 1)

        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx

In [ ]:
start_context = "Hello, I am"

encoded = tokenizer.encode(start_context)
print("encoded:", encoded)

encoded_tensor = torch.tensor(encoded).unsqueeze(0)
print("encoded_tensor.shape:", encoded_tensor.shape)

encoded: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])


In [ ]:
model.eval() # disable dropout

out = generate_text_simple(
    model=model,
    idx=encoded_tensor,
    max_new_tokens=6,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output:", out)
print("Output length:", len(out[0]))

Output: tensor([[15496,    11,   314,   716, 15676, 24377, 35069,  8671, 13947, 11262]])
Output length: 10


In [ ]:
with torch.no_grad():
    outputs = model(inputs)

print("Outputs:\n", outputs)
print("Outputs dimensions:", outputs.shape)

Outputs:
 tensor([[[ 0.3936, -1.0258, -0.2400,  ...,  0.5200, -0.6952, -0.3451],
         [ 0.0938,  0.6579,  0.8055,  ..., -0.8616,  0.0940, -0.0102],
         [ 0.5289,  0.1792, -0.2203,  ..., -0.7402, -0.4950, -1.0599],
         [ 0.2946, -0.6179,  0.1262,  ...,  0.3460, -1.3460, -0.2300]]])
Outputs dimensions: torch.Size([1, 4, 50257])


In [ ]:
for param in model.parameters():
    param.requires_grad = False
torch.manual_seed(123)

num_classes = 3
model.out_head = torch.nn.Linear(in_features=GPT_CONFIG_124M["emb_dim"], out_features=num_classes)

In [ ]:
for param in model.trf_blocks[-1].parameters():
    param.requires_grad = True

for param in model.final_norm.parameters():
    param.requires_grad = True

inputs = tokenizer.encode("Do you have time")
inputs = torch.tensor(inputs).unsqueeze(0)
print("Inputs:", inputs)
print("Inputs dimensions:", inputs.shape) # shape: (batch_size, num_tokens)
with torch.no_grad():
    outputs = model(inputs)

Inputs: tensor([[5211,  345,  423,  640]])
Inputs dimensions: torch.Size([1, 4])


In [ ]:
decoded_text = tokenizer.decode(out.squeeze(0).tolist())
print(decoded_text)

Hello, I am summar Amsterdam��tditzerlandAST


In [ ]:
print("Last output token:", outputs[:, -1, :])


Last output token: tensor([[ 0.6304, -0.6248,  0.3216]])


In [ ]:
probas = torch.softmax(outputs[:, -1, :], dim=-1)
label = torch.argmax(probas)
print("Class label:", label.item())

Class label: 0


# EVALUATE

A comprehensive set of evaluation metrics was defined to objectively assess the model's performance.

In [ ]:
def calc_loss_batch(input_batch, target_batch, model,
                    trainable_token_pos=-1, average_embeddings=False):
    input_batch, target_batch = input_batch, target_batch

    model_output = model(input_batch)
    if average_embeddings:
        # Average over the sequence dimension (dim=1)
        logits = model_output.mean(dim=1)
    else:
        # Select embeddings at the specified token position
        logits = model_output[:, trainable_token_pos, :]

    loss = torch.nn.functional.cross_entropy(logits, target_batch)
    return loss


def calc_loss_loader(data_loader, model,
                     num_batches=None, trainable_token_pos=-1,
                     average_embeddings=False):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(
                input_batch, target_batch, model,
                trainable_token_pos=trainable_token_pos, average_embeddings=average_embeddings
            )
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches


@torch.no_grad()  # Disable gradient tracking for efficiency
def calc_accuracy_loader(data_loader, model,
                         num_batches=None, trainable_token_pos=-1,
                         average_embeddings=False):
    model.eval()
    correct_predictions, num_examples = 0, 0

    if num_batches is None:
        num_batches = len(data_loader)
    else:
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            input_batch, target_batch = input_batch, target_batch

            model_output = model(input_batch)
            if average_embeddings:
                # Average over the sequence dimension (dim=1)
                logits = model_output.mean(dim=1)
            else:
                # Select embeddings at the specified token position
                logits = model_output[:, trainable_token_pos, :]

            predicted_labels = torch.argmax(logits, dim=-1)

            num_examples += predicted_labels.shape[0]
            correct_predictions += (predicted_labels == target_batch).sum().item()
        else:
            break
    return correct_predictions / num_examples


def evaluate_model(model, train_loader, val_loader, eval_iter,
                   trainable_token_pos=-1, average_embeddings=False):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(
            train_loader, model, num_batches=eval_iter,
            trainable_token_pos=trainable_token_pos, average_embeddings=average_embeddings
        )
        val_loss = calc_loss_loader(
            val_loader, model, num_batches=eval_iter,
            trainable_token_pos=trainable_token_pos, average_embeddings=average_embeddings
        )
    model.train()
    return train_loss, val_loss


# TRAIN


The train_classifier_simple function is defined to train a classification model, accepting the model, training and validation data loaders, optimizer, training parameters (epochs, evaluation frequency, etc.), and optional configuration options.

In [ ]:
def train_classifier_simple(model, train_loader, val_loader, optimizer, num_epochs,
                            eval_freq, eval_iter, max_steps=None, trainable_token_pos=-1,
                            average_embeddings=False):
    # Initialize lists to track losses and tokens seen
    train_losses, val_losses, train_accs, val_accs = [], [], [], []
    examples_seen, global_step = 0, -1

    # Main training loop
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode

        for input_batch, target_batch in train_loader:
            optimizer.zero_grad()  # Reset loss gradients from previous batch iteration
            loss = calc_loss_batch(input_batch, target_batch, model,
                                   trainable_token_pos=trainable_token_pos, average_embeddings=average_embeddings)
            loss.backward()  # Calculate loss gradients
            optimizer.step()  # Update model weights using loss gradients
            examples_seen += input_batch.shape[0]  # New: track examples instead of tokens
            global_step += 1

            # Optional evaluation step
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, eval_iter,
                    trainable_token_pos=trainable_token_pos, average_embeddings=average_embeddings
                )
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")

            if max_steps is not None and global_step > max_steps:
                break

        # New: Calculate accuracy after each epoch
        train_accuracy = calc_accuracy_loader(
            train_loader, model, num_batches=eval_iter,
            trainable_token_pos=trainable_token_pos, average_embeddings=average_embeddings
        )
        val_accuracy = calc_accuracy_loader(
            val_loader, model, num_batches=eval_iter,
            trainable_token_pos=trainable_token_pos, average_embeddings=average_embeddings
        )
        print(f"Training accuracy: {train_accuracy*100:.2f}% | ", end="")
        print(f"Validation accuracy: {val_accuracy*100:.2f}%")
        train_accs.append(train_accuracy)
        val_accs.append(val_accuracy)

        if max_steps is not None and global_step > max_steps:
            break

    return train_losses, val_losses, train_accs, val_accs, examples_seen

In [ ]:

# Ensure GPU is utilized
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

start_time = time.time()

torch.manual_seed(123)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.1)

num_epochs = 3  # Reduce epochs for testing
train_losses, val_losses, train_accs, val_accs, examples_seen = train_classifier_simple(
    model, train_loader, val_loader, optimizer,
    num_epochs=num_epochs, eval_freq=200, eval_iter=10,  # Evaluate less frequently
)

end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")


Ep 1 (Step 000000): Train loss 1.408, Val loss 1.438
Ep 1 (Step 000200): Train loss 1.118, Val loss 1.109
Ep 1 (Step 000400): Train loss 1.071, Val loss 1.100
Ep 1 (Step 000600): Train loss 1.102, Val loss 1.100
Ep 1 (Step 000800): Train loss 1.084, Val loss 1.102
Ep 1 (Step 001000): Train loss 1.101, Val loss 1.097
Ep 1 (Step 001200): Train loss 1.091, Val loss 1.108
Ep 1 (Step 001400): Train loss 1.096, Val loss 1.097


KeyboardInterrupt: 

# CLASSIFICATION HEAD

The train_classifier_simple function is defined to train a classification model, taking the model, data loaders, optimizer, hyperparameters, and optional configuration parameters as input.

In [ ]:
def classify_review(text, model, tokenizer, max_length=None, pad_token_id=50256):
    model.eval()

    # Prepare inputs to the model
    input_ids = tokenizer.encode(text)
    supported_context_length = model.pos_emb.pe.shape[0]
    # Note: In the book, this was originally written as pos_emb.weight.shape[1] by mistake
    # It didn't break the code but would have caused unnecessary truncation (to 768 instead of 1024)

    # Truncate sequences if they too long
    input_ids = input_ids[:min(max_length, supported_context_length)]

    # Pad sequences to the longest sequence
    input_ids += [pad_token_id] * (max_length - len(input_ids))
    input_tensor = torch.tensor(input_ids).unsqueeze(0) # add batch dimension

    # Model inference
    with torch.no_grad():
        logits = model(input_tensor)[:, -1, :]  # Logits of the last output token
    predicted_label = torch.argmax(logits, dim=-1).item()

    # Return the classified result
    return "negative" if predicted_label == 0 else "neutral" if predicted_label == 1 else "positive"

In [ ]:
text_1 = (
    "You are a winner you have been specially"
    " selected to receive $1000 cash or a $2000 award."
)

print(classify_review(
    text_1, model, tokenizer, max_length=train_dataset.max_length
))

spam
